# Dashboard v2

Este pequeño dashboard mostrará el número de pokemos que han aparecido en el sistema y los usuarios con sesión abierta. El procesado es muy sencillo, pero nos obliga a mantener un estado.

Este dashboard es idéntico al original en cuanto al tratamiento de streams, pero hemos añadido un simple servidor web que devuelve un pequeño HTML con los valores actuales de las métricas que recuperamos a partir de los streams. El servidor web arranca como un segundo thread, de manera que podemos, en un mismo proceso, mantener un estado (las métricas de pokemos y usuarios) y mostrarlo en una interfaz independiente de Kafka. De la misma manera que servimos un HTML simple podríamos ofrecer los datos en una API REST más sofisticada que alimentara una UI avanzada en, por ejemplo, Tableau, una app móvil o una applicación web.

In [ ]:
from confluent_kafka import Consumer, KafkaError
import time
import json

from ejercicios.pokemons import TOPIC_USERS, TOPIC_POKEMONS, GROUP_DASHBOARD

In [ ]:
pokemon_count = 0
user_count = 0

In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import threading
    
template = """<html>
<head>
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
</head>
<body>
<div class="ml-5 mt-5">
<h1>Pokemon GO</h1>
<div class="row">
  <div class="col-sm-2">Pokemons:</div>
  <div class="col-sm-10 font-weight-bold">{0}</div>
</div>

<div class="row">
  <div class="col-sm-2">Online users:</div>
  <div class="col-sm-10 font-weight-bold">{1}</div>
</div>
</div>
</body></html>
"""

class DashboardHandler(BaseHTTPRequestHandler):
    
    def do_GET(self):
        # Este método devuelve siempre el HTML sin tener en cuenta cabeceras ni path.
        # Es dinámico ya que renderizamos el template con las métricas cada vez
        # que servimos una petición.
        global pokemon_count
        global user_count
        self.send_response(200, "OK")
        self.end_headers()
        self.wfile.write(template.format(pokemon_count, user_count).encode())

def run_http():
    server_address = ('', 8000)
    httpd = HTTPServer(server_address, DashboardHandler)
    httpd.serve_forever()



In [ ]:
thread_out = threading.Thread(target=run_http)
thread_out.start()

Ahora podemos acceder al dashboard en http://192.168.53.3:8000/

In [ ]:
c = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': GROUP_DASHBOARD + "_web",
    'auto.offset.reset': 'earliest'
})
c.subscribe([TOPIC_USERS, TOPIC_POKEMONS])

while True:
    msg = c.poll(5.0)


    if msg is None:
        continue

    if msg.error():
        print("Consumer error: {}".format(msg.error()))
        continue

    if msg.topic() == TOPIC_POKEMONS:
        pokemon_count += 1
    elif msg.topic() == TOPIC_USERS:
        if msg.value() is None:
            user_count -= 1
        else:
            user_count += 1

    print("Pokemons: {} | Logged users: {}".format(pokemon_count, user_count))    
    
c.close()